<h1 align='center'><b><font color ='pickle'>Lecture Softmax Regression PyTorch nn.Module</b></h1>




# <Font color = 'pickle'>**Load Libraries/Install Software**

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
else:
  print('Not running on CoLab')

Running on CoLab


In [ ]:
# Install wandb and update it to the latest version
if 'google.colab' in str(get_ipython()):
    !pip install wandb --upgrade -q

     |████████████████████████████████| 1.8 MB 2.1 MB/s 
     |████████████████████████████████| 181 kB 63.7 MB/s 
     |████████████████████████████████| 162 kB 60.3 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 158 kB 60.9 MB/s 
     |████████████████████████████████| 157 kB 56.5 MB/s 
     |████████████████████████████████| 157 kB 71.8 MB/s 
     |████████████████████████████████| 157 kB 71.3 MB/s 
     |████████████████████████████████| 157 kB 63.2 MB/s 
     |████████████████████████████████| 157 kB 79.1 MB/s 
     |████████████████████████████████| 157 kB 63.1 MB/s 
     |████████████████████████████████| 157 kB 52.7 MB/s 
     |████████████████████████████████| 156 kB 73.2 MB/s 


In [ ]:
# mount google drive
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Importing the necessary libraries
import torch
import torch.nn as nn
import numpy as np
from pathlib import Path
import wandb
import spacy
import sys

import pandas as pd
import joblib

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import  TfidfVectorizer

In [ ]:
if 'google.colab' in str(get_ipython()):
  !python -m spacy download 'en_core_web_sm'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 2.1 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
# Login to W&B
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# <Font color = 'pickle'>**Specify Project Folders**

In [ ]:
# This is the path where we will downlaod and save data
if 'google.colab' in str(get_ipython()):
  base_folder = Path('/content/drive/MyDrive/data')
else:
  base_folder = Path('/home/harpreet/Insync/google_drive_shaannoor/data')

In [ ]:
data_folder = base_folder/'datasets/aclImdb'
model_folder = base_folder/'models/nlp_fall_2022/imdb'
custom_functions = base_folder/'custom-functions'

In [ ]:
sys.path.append(str(custom_functions))

In [ ]:
sys.path

['/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/MyDrive/data/custom-functions']

In [ ]:
import custom_preprocessor as cp

# <Font color = 'pickle'>**IMDB Dataset**

For this notebook, we will use IMDB movie review dataset. <br>
LInk for complete dataset: http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz.

We downloaded the dataset in the previous lecture 2 notebook (notebook: 3_Faster_tokenization_spacy_final.ipynb)

We created csv files in Lecture 2 -- train.csv and test.csv file. The files are availible in Lecture2/data folder from eLearning. I have applied the custom pre=processor and cleaned the data set for this lecture. I pickled the datasets and saved them as files. The files are available in Lecture_6/data folder. We will download the following files as well.

- 'x_train_cleaned_bag_of_words.pkl'
- 'x_valid_cleaned_bag_of_words.pkl'
- 'x_test_cleaned_bag_of_words.pkl'

In [ ]:
# location of train and test files
train_file = data_folder /'train.csv'
test_file = data_folder /'test.csv'

In [ ]:
# creating Pandas Dataframe
train_data = pd.read_csv(train_file, index_col=0)
test_data = pd.read_csv(test_file, index_col=0)

In [ ]:
# print shape of the datasets
print(f'Shape of Training data set is : {train_data.shape}')
print(f'Shape of Test data set is : {test_data.shape}')

Shape of Training data set is : (25000, 2)
Shape of Test data set is : (25000, 2)


In [ ]:
train_data.head()

,Reviews,Labels
0,Ever wanted to know just how much Hollywood co...,1
1,The movie itself was ok for the kids. But I go...,1
2,You could stage a version of Charles Dickens' ...,1
3,this was a fantastic episode. i saw a clip fro...,1
4,and laugh out loud funny in many scenes.<br />...,1


## <Font color = 'pickle'>**Create Train/Test/Valid Split**


In [ ]:
X, y = train_data['Reviews'].values, train_data['Labels'].values

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.20, random_state=42)

In [ ]:
X_test , y_test = test_data['Reviews'].values, test_data['Labels'].values

## <Font color = 'pickle'>**Data PreProcessing**

In [ ]:
# X_train_cleaned = cp.SpacyPreprocessor(model = 'en_core_web_sm', batch_size=1000).transform(X_train)

In [ ]:
# X_valid_cleaned = cp.SpacyPreprocessor(model = 'en_core_web_sm', batch_size=1000).transform(X_valid)
# X_test_cleaned = cp.SpacyPreprocessor(model = 'en_core_web_sm', batch_size=1000).transform(X_test)

In [ ]:
X_train_cleaned_file = data_folder / 'x_train_cleaned_bag_of_words.pkl'
X_valid_cleaned_file = data_folder / 'x_valid_cleaned_bag_of_words..pkl'
X_test_cleaned_file = data_folder / 'x_test_cleaned_bag_of_words..pkl'

In [ ]:
# joblib.dump(X_train_cleaned, X_train_cleaned_file)
# joblib.dump(X_valid_cleaned, X_valid_cleaned_file)
# joblib.dump(X_test_cleaned, X_test_cleaned_file)

In [ ]:
X_train_cleaned = joblib.load(X_train_cleaned_file)
X_valid_cleaned = joblib.load(X_valid_cleaned_file)
X_test_cleaned = joblib.load(X_test_cleaned_file)

In [ ]:
from sklearn.feature_extraction.text import  TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words="english", min_df=4, max_features=5000).fit(X_train_cleaned)

X_train_vec = vectorizer.transform(X_train_cleaned)
X_valid_vec = vectorizer.transform(X_valid_cleaned)
X_test_vec = vectorizer.transform(X_test_cleaned)

In [ ]:
# Creating a tensors of data

X_train_tensor = torch.tensor(X_train_vec.toarray()).float()
X_valid_tensor = torch.tensor(X_valid_vec.toarray()).float()
X_test_tensor = torch.tensor(X_test_vec.toarray()).float()

y_train_tensor = torch.tensor(np.array(y_train)).long()
y_valid_tensor = torch.tensor(np.array(y_valid)).long()
y_test_tensor = torch.tensor(np.array(y_test)).long()

## <Font color = 'pickle'>**Create Tensor Dataset**

In [ ]:
# Creating tensor dataset
train_set = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
valid_set = torch.utils.data.TensorDataset(X_valid_tensor, y_valid_tensor)
test_set = torch.utils.data.TensorDataset(X_test_tensor, y_test_tensor)

## <Font color = 'pickle'>**Check inputs**

In [ ]:
# Shape of training data
train_set.tensors[0].shape

torch.Size([20000, 5000])

In [ ]:
# shape of valid data
valid_set.tensors[0].shape

torch.Size([5000, 5000])

In [ ]:
# Shape of testing data
test_set.tensors[0].shape

torch.Size([25000, 5000])

In [ ]:
# Sahpe Target values
train_set.tensors[1].shape

torch.Size([20000])

In [ ]:
# Unique Target values
train_set.tensors[1].unique()

tensor([0, 1])

# <Font color = 'pickle'>**Data Loaders**

The next step is to create dataloader for train, valid, and test set using pytorch dataloader. This will be an iterator that will provide dataset in batches.  Let's keep batch size as 256. 

In [ ]:
# Initializing the batch size
batch_size = 256

# Creating data loader for train set
train_loader = torch.utils.data.DataLoader(dataset = train_set, 
                                            batch_size = batch_size, 
                                            shuffle = True)

valid_loader = torch.utils.data.DataLoader(dataset = valid_set, 
                                            batch_size = batch_size, 
                                            shuffle = False)

# Creating data loader for test set
test_loader = torch.utils.data.DataLoader(dataset = test_set, 
                                          batch_size = batch_size, 
                                          shuffle = False)

In [ ]:
# check number of batches
len(train_loader)

79

In [ ]:
# check total training examples
len(train_loader.dataset)

20000

In [ ]:
# check imputs and outputs 
for input_, target in train_loader:
  print(f'shape of inputs is :{input_.shape}')
  print(f'\nmax input value  :{input_.max()}')
  print(f'\nmin input value  :{input_.min()}')
  print(f'\nmean input value  :{input_.mean()}')
  print(f'\nstd input value  :{input_.std()}')
  print(f'\nshape of targets is :{target.shape}')
  print(f'\ninputs  :{input_[0:10]}')
  
  break

shape of inputs is :torch.Size([256, 5000])

max input value  :0.8381882905960083

min input value  :0.0

mean input value  :0.0013669105246663094

std input value  :0.014075926505029202

shape of targets is :torch.Size([256])

inputs  :tensor([[0.0000, 0.0000, 0.1052,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0432,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])


# <font color = 'pickle'> **Functions to implement Softmax**

Now, we will start implementing our Softmax Regression Model from scratch.

We will now create following functions:

- **Model**
- **Loss Function** 
- **One Hot Encoding**
- **Training Loop for 1 epoch**
- **Validation Loop for 1 epoch**
- **Model Training** - repeat the training and validation loops for given number of epochs
- **Function to get the accuracy given the model**

## <Font color = 'pickle'>**Model**

In [ ]:
import torch.nn as nn

In [ ]:
def softmax_reg(n_ins, n_outs, bias):
  return nn.Sequential(nn.Linear(n_ins, n_outs))

## <Font color = 'pickle'>**Loss Function**

In [ ]:
loss_function = nn.CrossEntropyLoss()

## <Font color = 'pickle'>**Function for Training  Loops**

**Model Training** involves five steps: 

- Step 0: Randomly initialize parameters / weights
- Step 1: Compute model's predictions - forward pass
- Step 2: Compute loss
- Step 3: Compute the gradients
- Step 4: Update the parameters
- Step 5: Repeat steps 1 - 4

Model training is repeating this process over and over, for many **epochs**.

We will specify number of ***epochs*** and during each epoch we will iterate over the complete dataset and will keep on updating the parameters.

***Learning rate*** and ***epochs*** are known as hyperparameters. We have to adjust the values of these two based on validation dataset.

We will now create functions for step 1 to 4.

In [ ]:
def train(train_loader, learning_rate, loss_function, model, optimizer):

  """ 
  Function for training the model in each epoch
  Input: iterator for train dataset, weights and bias, number of outputs, learning rate,
  loss function, model.
  Output: weights, bias, train loss, train accuracy at end of each epoch
  """

  # Step 0: Randomly initialize parameters / weights - We give this as input to function for first epoch
  # In subsequent epochs the function gets the updated weights from last epoch

  # Training Loop 
  # Initialize train_loss at the he start of the epoch
  running_train_loss = 0
  running_train_correct = 0
  
  # Iterate on batches from the dataset using train_loader
  for input_, targets in train_loader:
    
    # move inputs and outputs to GPUs
    input_ = input_.to(device)
    targets = targets.to(device)


    # Step 1: Forward Pass: Compute model's predictions 
    output = model(input_)
    
    # Step 2: Compute loss
    loss = loss_function(output, targets)

    # Correct prediction
    y_pred = torch.argmax(output, dim = 1)
    correct = torch.sum(y_pred == targets)

    # Step 3: Backward pass -Compute the gradients
    optimizer.zero_grad()
    loss.backward()

    # Step 4: Update the parameters
    optimizer.step()
          
    # Add train loss of a batch 
    running_train_loss += loss.item()

    # Add Corect counts of a batch
    running_train_correct += correct
  
  # Calculate mean train loss for the whole dataset for a particular epoch
  train_loss = running_train_loss/len(train_loader)

  # Calculate accuracy for the whole dataset for a particular epoch
  train_acc = running_train_correct/len(train_loader.dataset)
  

  return train_loss, train_acc

## <Font color = 'pickle'>**Function for Validation Loops**


In [ ]:
def validate(valid_loader, loss_function, model):

  """ 
  Function for calcuilating loss and prediction for validataion dataset. 
  Input: iterator for validation dataset, estimated weights and bias at the end of epoch in training loop, 
  learning rate,   loss function, model
  Output: val loss and accuracy for each epoch.
  """

  # Validation loop
  # Initialize valid_loss at the he strat of the epoch
  
  running_val_loss = 0
  running_val_correct = 0
    
  with torch.no_grad():
    for input_,targets in valid_loader:

      # move inputs and outputs to GPUs
      input_ = input_.to(device)
      targets = targets.to(device)

      # Step 1: Forward Pass: Compute model's predictions 
      output = model(input_)

      # Step 2: Compute loss
      loss = loss_function(output, targets)

      # Correct Predictions
      y_pred = torch.argmax(output, dim = 1)
      correct = torch.sum(y_pred == targets)

      # Add val loss of a batch 
      running_val_loss += loss.item()

      # Add correct count for each batch
      running_val_correct += correct

    # Calculate mean val loss for the whole dataset for a particular epoch
    val_loss = running_val_loss/len(valid_loader)

    # Calculate accuracy for the whole dataset for a particular epoch
    val_acc = running_val_correct/len(valid_loader.dataset)
    
  return val_loss, val_acc

## <Font color = 'pickle'>**Function for Model Training**
    
We will now create a function for step 5 of model training


In [ ]:
def train_loop(train_loader, valid_loader, model, optimizer, loss_function, epochs, device):
    
  """ 
  Function for training the model and plotting the graph for train & validation loss vs epoch.
  Input: iterator for train dataset, initial weights and bias, epochs, learning rate, batch size.
  Output: final weights, bias and train loss and validation loss for each epoch.
  """

  # Create lists to store train and val loss at each epoch
  train_loss_history = []
  valid_loss_history = []
  train_acc_history = []
  valid_acc_history = []

  # Iterate for the given number of epochs
  # Step 5: Repeat steps 1 - 4

  for epoch in range(epochs):

    # Get train loss and accuracy for one epoch
    train_loss, train_acc = train(train_loader, learning_rate, loss_function, model, optimizer)
    valid_loss, valid_acc   = validate(valid_loader, loss_function, model)

    # Save history of the Losses and accuracy
    train_loss_history.append(train_loss)
    train_acc_history.append(train_acc)

    valid_loss_history.append(valid_loss)
    valid_acc_history.append(valid_acc)

    # Log the train and valid loss to wandb
    wandb.log({f"Train Loss :": train_loss})
    wandb.log({f"Train Acc :": train_acc})

    wandb.log({f"Valid Loss :": valid_loss})
    wandb.log({f"Valid Acc :": valid_acc})


    # Print the train loss and accuracy for given number of epochs, batch size and number of samples
    print(f'Epoch : {epoch+1} / {epochs}')
    print(f'Train Loss: {train_loss : .4f} | Train Accuracy: {train_acc * 100 : .4f}%')
    print(f'Valid Loss: {valid_loss : .4f} | Valid Accuracy: {valid_acc * 100 : .4f}%')
    print()

  return train_loss_history, train_acc_history, valid_loss_history, valid_acc_history

## <Font color = 'pickle'>**Function for Accuracy and Predictions**

Now we have final values for weights and bias after training the model. We will use these values to make predictions on the test dataset.

In [ ]:
def get_acc_pred(data_loader, model, device):
    
  """ 
  Function to get predictions and accuracy for a given data using estimated model
  Input: Data iterator, Final estimated weoights, bias
  Output: Prections and Accuracy for given dataset
  """

  # Array to store predicted labels
  predictions = torch.Tensor() # empty tensor
  predictions = predictions.to(device) # move predictions to GPU

  # Array to store actual labels
  y = torch.Tensor() # empty tensor
  y = y.to(device)

  # Iterate over batches from data iterator
  with torch.no_grad():
    for input_, targets in data_loader:
      
      # move inputs and outputs to GPUs
      
      input_ = input_.to(device)
      targets = targets.to(device)
      
      # Calculated the predicted labels
      output = model(input_)

      # Choose the label with maximum probability
      prediction = torch.argmax(output, dim = 1)

      # Add the predicted labels to the array
      predictions = torch.cat((predictions, prediction)) 

      # Add the actual labels to the array
      y = torch.cat((y, targets)) 

  # Check for complete dataset if actual and predicted labels are same or not
  # Calculate accuracy
  acc = (predictions == y).float().mean()

  # Return tuple containing predictions and accuracy
  return predictions, acc  

# <Font color = 'pickle'>**Hyperparameters**

In [ ]:
# Initialize a new project
wandb.init(name = "imdb_softmax", project = 'nlp22_L6')

# Initialize number of epochs, learning rate and batch size
learning_rate = 0.5
wandb.log({'learning_rate': learning_rate})
epochs = 50

# device 
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Initialize number of inputs (features: 5000) and outputs (labels: 2)
num_inputs = train_set.tensors[0].shape[1]
num_outputs = 2

# Initialize weights from normal distribution with mean 0 and standard deviation 0.01
def init_weights(layer):
  if type(layer) == nn.Linear:
    torch.nn.init.normal_(layer.weight, mean = 0, std = 0.015)
    torch.nn.init.zeros_(layer.bias)

# loss_function
loss_function = loss_function

model = softmax_reg(num_inputs, num_outputs, bias = True)
model.to(device)
model.apply(init_weights)

optimizer = torch.optim.SGD(model.parameters(), learning_rate)

wandb: Currently logged in as: hsingh-utd. Use `wandb login --relogin` to force relogin


# <Font color = 'pickle'>**Training Model**

In [ ]:
# See live graphs in the notebook.
#%%wandb 
train_loss_history, train_acc_history, valid_loss_history, valid_acc_history = train_loop(train_loader, 
                                                                                          valid_loader, 
                                                                                          model, 
                                                                                          optimizer, 
                                                                                          loss_function, 
                                                                                          epochs, 
                                                                                          device)

Epoch : 1 / 50
Train Loss:  0.6772 | Train Accuracy:  68.1500%
Valid Loss:  0.6613 | Valid Accuracy:  78.0800%

Epoch : 2 / 50
Train Loss:  0.6465 | Train Accuracy:  79.2300%
Valid Loss:  0.6342 | Valid Accuracy:  76.9400%

Epoch : 3 / 50
Train Loss:  0.6209 | Train Accuracy:  80.9300%
Valid Loss:  0.6112 | Valid Accuracy:  78.4400%

Epoch : 4 / 50
Train Loss:  0.5985 | Train Accuracy:  81.6500%
Valid Loss:  0.5908 | Valid Accuracy:  80.3200%

Epoch : 5 / 50
Train Loss:  0.5791 | Train Accuracy:  82.2500%
Valid Loss:  0.5735 | Valid Accuracy:  82.6800%

Epoch : 6 / 50
Train Loss:  0.5626 | Train Accuracy:  82.6100%
Valid Loss:  0.5583 | Valid Accuracy:  83.0800%

Epoch : 7 / 50
Train Loss:  0.5475 | Train Accuracy:  83.1600%
Valid Loss:  0.5440 | Valid Accuracy:  82.3200%

Epoch : 8 / 50
Train Loss:  0.5339 | Train Accuracy:  83.3350%
Valid Loss:  0.5316 | Valid Accuracy:  82.9000%

Epoch : 9 / 50
Train Loss:  0.5222 | Train Accuracy:  83.7350%
Valid Loss:  0.5205 | Valid Accuracy:  82

We can observe that with each epoch, our loss is getting reduced.

# <Font color = 'pickle'>**Get Accuracy, Predictions**

In [ ]:
device

device(type='cuda', index=0)

In [ ]:
# Get the prediction and accuracy for the test dataseta
predictions_test, acc_test = get_acc_pred(test_loader, model, device)

In [ ]:
# Print Test Accuracy
print('Test accuracy', acc_test * 100)

Test accuracy tensor(85.7960, device='cuda:0')


In [ ]:
wandb.log({'Test_Acc': acc_test})

# <Font color = 'pickle'>**Confusion Matrix for Test Data**

Now, we will make some visualizations for the predictions that we obtained.

We will construct a `confusion matrix` which will help us to visualize the performance of our classification model on the test dataset as we know the true values for the test data.

In [ ]:
test_set.tensors[1].shape

torch.Size([25000])

In [ ]:
# Get an array containing actual labels
testing_labels = np.array(test_set.tensors[1])

In [ ]:
np.unique(testing_labels)

array([0, 1])

In [ ]:
# Log a confusion matrix to W&B
wandb.log({"conf_mat" : wandb.plot.confusion_matrix(
                        probs = None,
                        y_true = testing_labels,
                        preds = predictions_test.to('cpu').numpy(),
                        class_names =['negative', 'positive'])})

In [ ]:
wandb.finish()

Test_Acc,▁
Train Acc :,▁▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██████████████████
Train Loss :,█▇▇▆▆▅▅▅▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
Valid Acc :,▂▁▂▃▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇█████████
Valid Loss :,█▇▇▆▆▅▅▅▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
learning_rate,▁
Test_Acc,0.85796
Train Acc :,0.87815
Train Loss :,0.35523
Valid Acc :,0.8702
Valid Loss :,0.36734
